# Working with NetCDF files

**In atmospheric science, we have a special data file format called NetCDF (.nc).** This specially designed data file format allows gridded atmospheric data to be saved easily with attributes and coordinates attached to each of the variables. Before we learn how to process them, let's take a quick look on how Python handle files. 

## General idea on file inputs and outputs
Suppose you want to read or write a text/binary file in your script, you will need the following functions to help you:
- open(*filename* \[, *access_mode*\]): Opens a file with the given *filename* under an access mode, e.g. "r" means read-only access, "w" means write-only access, and return the file object.

Syntax: 
```Python
fileObject = open(filename) # By default the access is read-only
```
- *fileObject*.read(*number*): Read the *number* of characters in the file (referenced by the file object).
- *fileObject*.readline( ): Read one line from the file and move to the next line.
- *fileObject*.write(*string*): Write the *string* to file.
- *fileObject*.close( ): Close the file.

These functions should have similar functionality in different languages, so you probably have learnt them before, and we will skip the details here.

## Using xarray to process NetCDF files
NetCDF files cannot be easily read using the default methods above. However, the good news is someone has already written relevant modules to allow users to read/write .nc files relatively easily. These packages can be easily installed when creating a new conda environment. I have provided the [environment.txt](./environment.txt) file, which specifies a list of packages to be used. Follow the installation guide for [Mac](./pre_lab_installation_guide_mac.md#Install-conda-environment-for-extra-packages)/[Windows](./pre_lab_installation_guide_Windows.md#Install-conda-environment-for-extra-packages) to install them.


The module we are going to use is called "xarray".

### Input from NetCDF files

In [ ]:
# import library to read .nc files
import xarray as xr
# You can check that this can only be done inside the environment, but not outside.

In [ ]:
# Open the dataset using the library
ds = xr.open_dataset('./misc/sample_data.nc')
print(type(ds)) # Note that this is a class called Dataset
print(ds)

Here we have just opened our data file, and `ds` is an xarray Dataset object (hereafter **Dataset**). It contains all information of a NetCDF file. To use it, just think of `ds` as a dictionary, and apply the dictionary syntax (recall [Part 1: Dictionary](./Part1_Basic_Syntax.ipynb#How-to-access-values-in-a-dictionary) ):
```
value = dictionary['key']
```
where the *key* is the name of variable here.

### Extracting a variable
Let's get the temperature variable `temp` from the dataset. 

In [ ]:
da = ds['temp']
print(type(da)) # Note that this is a class called DataArray
print(da)

`da` is an xarray DataArray object (hereafter **DataArray**), which behaves like a NumPy array. Therefore you can use NumPy syntax to operate on this array (Learn them here: [Part 1b: NumPy Arrays](./Part1b_NumPy_Arrays.ipynb) or ask Google). However, it is functionally more than just NumPy array. You will see that there are **attributes** (sometimes called metadata) and **coordinates** attached to this object (Recall [Part 4: Objects](./Part4_OOP.ipynb#Attributes-and-Methods)). (You may also look at the attributes and coordinates of a **Dataset**.)

### Output to netCDF files
You may want to save a **Dataset** or **DataArray** to a netCDF file. They can be saved in a similar fashion.

**Syntax:**
```
DatasetObject.to_netcdf(filename)
DataArrayObject.to_netcdf(filename)
```

In [ ]:
# Example
ds.to_netcdf('dataset_to_nc.nc')
da.to_netcdf('dataarray_to_nc.nc')

You can then check the current directory to see the .nc you just saved.

## Attributes
Suppose you want to access the attributes.

In [ ]:
print(da.attrs) 
# Note that these are attributes, not methods, so you don't need a pair of brackets after .attrs

In [ ]:
print(da.attrs)

This is again a dictionary. Again you can use the dictionary syntax for accessing and modifying the attributes.

In [ ]:
# Example for accessing attributes
print(da.attrs['long_name'])

In [ ]:
# Example for adding new attribute
da.attrs['new_attribute'] = 'This attribute has no meaning.'
print(da.attrs)

Suppose you want to change the unit from K to $^\circ$C. Then you will need to subtract the whole array by 273.15 and also change the *units* in attribute.

In [ ]:
# Example for modifying attrbutes
da_degC = da - 273.15
print(da_degC)

For whatever reason, the attributes won't be copied automatically to the new array. Let's do it manually

In [ ]:
da_degC.attrs = da.attrs 
# Now we change the 'units' in attributes
da_degC.attrs['units'] = 'degC'
print(da_degC)
# You can see that the 'valid range' and 'actual range' should also be changed. 
# I changed the 'units' only for demonstration purpose.

## Coordinates
You may access the coordinates from either the **Dataset** or the **DataArray**, again using dictionary syntax.

In [ ]:
print(ds.coords['lat'])

In [ ]:
print(da.coords['lat'])

In [ ]:
# You may check to see if they are identical or not.
all(ds.coords['lat'] == da.coords['lat']) 

And once again, each of the coordinates itself is an **DataArray**, so you may want to [revisit the syntax again](#Extracting-a-variable).

## Commonly used functions
Commonly used functions/methods include: 
- `all()`, `any()`, `floor()`, `ceil()` (Python pre-defined functions)
- `sqrt()`,`sin()`, `cos()`, `tan()`, `exp()`, `log()`, `log10()`, `isnan()`, `isinf()`, `isfinite()` (NumPy functions)
- `max()`, `min()`, `mean()`, `median()`, `sum()`, `std()`, `var()`, `cumsum()`, `where()`, `isin()` (DataArray methods)

Here we highlight the usage of a few DataArray methods.

In [ ]:
# Check whether the input elements exist in the DataArray
print(da.coords['lat'].isin([30,31,32,33,34,35]))

In [ ]:
# Grabs the DataArray with latitude > 30N
print(da.where(da.coords['lat']>30, drop=True))

In [ ]:
# zonal and meridional average
print(da.mean(dim=['lat','lon']))

## Index slicing
As mentioned above, you can apply (most of) NumPy syntax, e.g. index slicing, on **DataArray**.

In [ ]:
# Let's look at the temperature data again
print(da)

Suppose you want the values for the lowest model level, in this case 1000hPa.

In [ ]:
print(da[:,0,:,:])

However, this is not very convenient if you don't know the coordinates at the first place (in other words, you need to know the coordinates first). 

Instead the xarray DataArray object offers a **grid-independent** *method* of index slicing. That means you only need to know what coordinates the array has (but not the order of coordinates), and you can grab a slice of the array that you need. 

**Syntax**: (**SEL**ect by coordinate name and **I**ndex)
```
values = DataArrayObject.isel(coordinate_name = coordinate_index)
```
**Syntax**: (**SEL**ect by coordinate name and value)
```
values = DataArrayObject.sel(coordinate_name = coordinate_value)
```

However, you should **NOT** edit the **DataArray** directly using these two methods.

In [ ]:
# Again, let's choose the lowest model level
print(da.isel(level=0))

In [ ]:
# Get the data at 30N,60E
print(da.sel(lat=30,lon=60))

This is useful, but if you try other combinations, which the values may be absent from the coordinate array, then you will get an error. 

Fortunately, the developers have prepared for this situation. You can use an extra input argument `method='nearest'` to choose the data at the nearest coordinate.

In [ ]:
print(da.sel(lat=22.3,lon=114,method='nearest'))

Does these methods work with multiple indices/values? Let's try them.

In [ ]:
# Well, this is the Pythonic way to do index slicing, so it should work very fine.
print(da[:4,:,:36,:])

Next, let's try the `isel` method:

In [ ]:
# You may try using similar syntax, e.g. 0:3, as indices
da.isel(lat=0:3)
# However you will see that it does not work. 
# It's because 0:3 is not a valid syntax that gives a value for input arguments. (Note that isel is a method/function)

In [ ]:
# Instead you can use slice objects
# e.g. if you want the first 36 latitude indices and the first 4 time indices
print(da.isel(lat=slice(0,36), time=slice(0,4)))

Before using the `sel()` or `isel()` methods, it seems clearer to first organize the indices using a dictionary.

In [ ]:
print(da.sel(lat=slice(30,60.1,-1),lon=slice(0,180.1)))
# Note that a stride of -1 is needed here because the latitudes are decreasing.

### Time coordinate handling
You may have noticed that the time coordinate has a data type called *datetime64*. Let's learn how we can manipulate it. We will start off by looking at one of the dependencies of xarray, namely **pandas**. This is also a very popular library in data science.

In [ ]:
import pandas as pd
dates = pd.date_range(start='2019-01-14 06:00',periods=5,freq='6H')
print(dates)
print(dates+1) # every index shift by 1 period (6 hours)

In [ ]:
dates_new = pd.date_range(start='2019-01-16 06:00',end='20190131',freq='D')
print(dates_new)

There are several things I should mention here. 
1. date/time format: strings that looks like a date, e.g. `'2017/01/01 00:00:00'`, `'2017-01-01'`, `'20170101`, etc. Try what you think should work.
1. `start` and `end` arguments: the date/time denoting the start and end of the list (DatetimeIndex). Unlike Python lists, both ends are inclusive (although an extra argument can change that).
1. `freq` argument: a string denoting the difference between two indices. Commonly used strings: 
    - AS: year start frequency
    - MS: month start frequency
    - W: weekly frequency
    - D: daily frequency
    - H: hourly frequency 
    
   An integer in front would denote the number of multiples of it, e.g. `'6H'` means 6-hourly.
1. `periods` argument: an integer number of `freq` to repeat.

This is not the end of the datetime magic! The **DatetimeIndex** contains attributes and methods which allow you to quickly convert the dates to months,days of year, days of week, etc.

In [ ]:
# Attributes
print(dates.year)
print(dates.month)
print(dates.day)
print(dates.hour)
print(dates.dayofyear)
print(dates.dayofweek) # index starts at 0

In [ ]:
# Methods
print(dates.month_name())
print(dates.day_name())

Moreover, in xarray, `time.season` is also available (`'DJF'`/`'MAM'`/`'JJA'`/`'SON'`).

So now let's use similar framework to work with **DataArray** time coordinates. Firstly, let's look at the `isel()` and `sel()` methods.

In [ ]:
# Get the 2nd and 3rd value
print(da.isel(time=[1,2]))

When it comes to the `sel()` method, you can provide a 'datetime' formatted string (which grabs all indices that fit the string), or a **DatetimeIndex**.

In [ ]:
# Grab the values on the day 09 June 2017
print(da.sel(time='2017-06-09'))

In [ ]:
# Grab the values on the day 09 June 2017 using DatetimeIndex
date = pd.date_range(start='2017-06-09',periods=4,freq='6h')
print(da.sel(time=date))

We can also make use of the attributes and methods. Suppose we only want to look at Wednesday (`dayofweek==2`).

In [ ]:
# A boolean mask array using the attribute
print(da.coords['time.dayofweek']==2)

In [ ]:
# Use the mask array as index 
print(da[da.coords['time.dayofweek']==2])

Our **Dataset** is not really a good example to show the use of attribute `season`, but the following shows how you may use it.

In [ ]:
da[ds.coords['time.season']=='JJA']

### Groupby object
You can group up the array by some specific criterion and do operations easily. For example, you can take the daily mean with the following.

In [ ]:
da.groupby('time.dayofyear').mean(dim='time')

### Rolling object
Example: you can easily calculate moving average using *rolling* objects

In [ ]:
da.groupby('time.dayofyear').mean(dim='time').rolling(dayofyear=3).mean()

### Plotting DataArray on a map

Now you can combine the **DataArray** handling skills with plotting techniques that you learnt in previous part ([quick revision](./Part5_Plotting.ipynb#2D-Field-plots)) to make plots on a map.

Example:

In [ ]:
# import libraries
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
# Create the plotting longitudes by appending lon=360 to the end
# This will remove empty white column on the map
fill = xr.DataArray([360],
                    coords={'lon': [360]},
                    dims=['lon'],
                   )
plot_lon = xr.concat([da.coords['lon'],fill],dim='lon')
plot_lat = da.coords['lat']
print(plot_lon)

In [ ]:
# Create a new figure object
fig = plt.figure(figsize=[8, 4])
# Create a new axes object with PlateCarree projection
ax = plt.axes(projection=ccrs.PlateCarree())
# make the map global rather than have it zoom in to
# the extents of any plotted data
ax.set_global()
# Draw the coastlines
ax.coastlines() # There might be download warnings the first time you use this.
# Draw the dashed gridlines and axis labels
gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True,
                  linestyle='--',
                 )
# Disable labels on top and right axis
gl.xlabels_top = False
gl.ylabels_right = False
# Format the labels instead of showing numbers only
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# Next you can do the plotting
plot = ax.pcolormesh(plot_lon,plot_lat,da.isel(time=0,level=0),
                     cmap="RdBu_r",
                    )
fig.colorbar(plot, 
             ax=ax, 
#             orientation='horizontal',
             label='$T(^\circ C)$'
            )
ax.set_title('Air temperature at 1000hPa')
# save the figure
# plt.savefig('Airtemp_1000.png',dpi=160)

More examples on the [Research Computing in Earth Sciences series](https://rabernat.github.io/research_computing/intro-to-basemap.html) 

Remark: They are using Basemap, which is the ancestor of cartopy. I think you can just change the module name and things should work.